In [ ]:
import IPython.display as ipd
import os
import time
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from IPython.display import Audio, display
from concurrent.futures import ThreadPoolExecutor
import random
from collections import defaultdict
import pandas as pd
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential


In [ ]:
from keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score

In [2]:
main_directory='C:\Users\משתמש\Documents\תשפד\פרקטיקום'

In [ ]:
#הכנסת כל הקבצים לרשימה
file_lengths= []

for root, dirs, files in os.walk(main_directory):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if file_path.endswith(('.wav', '.mp3')):
            try:
                audio, sr = librosa.load(file_path)
                duration = librosa.get_duration(y=audio, sr=sr)
                file_lengths.append((file_name, duration))
                print(f"Loaded {file_path} with duration {duration} seconds")
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")

In [ ]:
#התפלגות ההקלטות לפניי המחיקה 
filtered_audio_lengths = [length for _,length in file_lengths if length > 0]
max_length = max(filtered_audio_lengths)
bins = list(range(0, int(max_length) + 10, 10))
plt.figure(figsize=(10, 6))
plt.hist(filtered_audio_lengths, bins=bins, color='skyblue', edgecolor='black')
plt.xlabel('Audio Length (seconds)')
plt.ylabel('Count')
plt.title('Distribution of Audio Lengths above 0 seconds')
plt.show()

In [6]:
#חלוקת ההקלטות לפי אורכם 
short_files = []  # רשימה לאחסון הקבצים שאורכם פחות מ-20 שניות
long_files = []   # רשימה לאחסון הקבצים שאורכם גדול מ-40 דקות

# Traverse the directory to load audio files and get their durations
for root, dirs, files in os.walk(main_directory):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if file_path.endswith(('.wav', '.mp3')):
            try:
                audio, sr = librosa.load(file_path)
                duration = librosa.get_duration(y=audio, sr=sr)
                if duration < 20:
                    short_files.append((file_name, duration, file_path))
                elif duration > 2400:  # 2400 seconds = 40 minutes
                    long_files.append((file_name, duration, file_path))
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")

In [ ]:
# Print the list of short files
print("\nList of audio files less than 20 seconds:")
for file_name, duration, file_path in short_files:
    print(f"{file_name}: {duration:.2f} seconds")

# Print the list of long files with colored output
print("\nList of audio files more than 40 minutes:")
for file_name, duration, file_path in long_files:
    print(f"{file_name}: {duration:.2f} seconds")

# Play the first 10 audio files that are less than 20 seconds long
print("\nPlaying the first 10 audio files less than 20 seconds:\n")
for file_name, duration, file_path in short_files[:10]:
    print(f"Playing: {file_path} ({duration:.2f} seconds)")
    display(ipd.Audio(file_path))

In [ ]:
#מחיקת ההקלטות הקצרות והארוכות
for file_name, duration, file_path in short_files:
    try:
        os.remove(file_path)  # הסרת הקובץ מהמערכת
        print(f"Removed file: {file_name}, Duration: {duration} seconds, Path: {file_path}")
    except Exception as e:
        print(f"Failed to remove {file_path}: {e}")
for file_name, duration, file_path in long_files:
    try:
        os.remove(file_path)  # הסרת הקובץ מהמערכת
        print(f"Removed file: {file_name}, Duration: {duration} seconds, Path: {file_path}")
    except Exception as e:
        print(f"Failed to remove {file_path}: {e}")      

In [9]:
# בניית רשימת file_lengths מחדש אחרי המחיקה
file_lengths = []
for root, dirs, files in os.walk(main_directory):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if file_path.endswith(('.wav', '.mp3')):
            try:
                audio, sr = librosa.load(file_path)
                duration = librosa.get_duration(y=audio, sr=sr)
                file_lengths.append((file_name, duration))
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")

In [ ]:
#התפלגות ההקלטות אחרי המחיקה 
plt.clf()
filtered_audio_lengths = [length for _,length in file_lengths if length > 0]
max_length = max(filtered_audio_lengths)
bins = list(range(0, int(max_length) + 10, 10))
plt.figure(figsize=(10, 6))
plt.hist(filtered_audio_lengths, bins=bins, color='skyblue', edgecolor='black')
plt.xlabel('Audio Length (seconds)')
plt.ylabel('Count')
plt.title('Distribution of Audio Lengths above 0 seconds')
plt.show()


In [ ]:
new_file_lengths= []

for root, dirs, files in os.walk(main_directory):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if file_path.endswith(('.wav', '.mp3')):
            try:
                audio, sr = librosa.load(file_path)
                duration = librosa.get_duration(y=audio, sr=sr)
                new_file_lengths.append((file_name, duration,file_path))
                print(f"Loaded {file_path} with duration {duration} seconds")
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")

In [ ]:
# Load the wav files
file1 = r"C:\09\12\1694520577.7685.mp3"
# Extract MFCC features
y1, sr1 = librosa.load(file1)
mfcc1 = librosa.feature.mfcc(y=y1, sr=sr1)
# Plot MFCC graph for file1
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc1, x_axis='time')
plt.colorbar()
plt.title('MFCC for ' + file1)
plt.tight_layout()
plt.show()
file2 = r"C:\09\11\1694443981.7528.mp3"
y2, sr2 = librosa.load(file2)
mfcc2 = librosa.feature.mfcc(y=y2, sr=sr2)
# Plot MFCC graph for file2
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc2, x_axis='time')
plt.colorbar()
plt.title('MFCC for ' + file2)
plt.tight_layout()
plt.show()

In [ ]:
# partition to clusters

In [ ]:
# Directory containing audio files
audio_dir = 'C:\\09'
audio_files = []
for root, dirs, files in os.walk(main_directory):
    for file in files:
        audio_files.append(os.path.join(root, file))
print(len(audio_files))

In [14]:
# Feature extraction function
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

In [15]:
# Extract features
features = [extract_mfcc(file) for file in audio_files]

In [16]:
features_np = np.array(features)

In [17]:
# Dimensionality reduction
pca = PCA(n_components=2)
features_reduced = pca.fit_transform(features_np)

In [18]:
#  חלוקה ל-2
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(features_reduced)

In [ ]:
# Visualization
plt.scatter(features_reduced[:, 0], features_reduced[:, 1], c=labels, cmap='viridis')
plt.title('Clustering of Audio Files')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar()
plt.show()

In [ ]:
# הצגת הקבצים, הקבוצות ומשך השיחה
group1=0
for idx, file in enumerate(audio_files):
    label = labels[idx]
    duration = librosa.get_duration(filename=file)
    print(f"File: {file} - Cluster: {label} - Duration: {duration:.2f} seconds")
    if label==0 :
       group1+=1
print(group1)     

In [21]:
# חלוקת קבצי האודיו לפי התוויות שנוצרו
clustered_audio_files = defaultdict(list)
for file, label in zip(audio_files, labels):
    clustered_audio_files[label].append(file)

In [ ]:
# בחירת 7 קבצים אקראיים מכל קבוצה
num_samples_per_cluster = 7
selected_files_per_cluster = {}
for cluster_id, files in clustered_audio_files.items():
    if len(files) <= num_samples_per_cluster:
        selected_files = files
    else:
        selected_files = random.sample(files, num_samples_per_cluster)    
    selected_files_per_cluster[cluster_id] = selected_files
# הצגת הקבצים שנבחרו והשמעתם
for cluster_id, selected_files in selected_files_per_cluster.items():
    print(f"\nCluster {cluster_id }:")
    for file in selected_files:
        display(Audio(filename=file))   

In [23]:
# Clustering
n_clusters = 4  # Changed number of clusters to 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(features_reduced)

In [ ]:
# Visualization
plt.scatter(features_reduced[:, 0], features_reduced[:, 1], c=labels, cmap='viridis')
plt.title('Clustering of Audio Files')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar()
plt.show()

In [ ]:
# הצגת הקבצים, הקבוצות ומשך השיחה
for idx, file in enumerate(audio_files):
    label = labels[idx]
    duration = librosa.get_duration(filename=file)
    print(f"File: {file} - Cluster: {label} - Duration: {duration:.2f} seconds")

In [26]:
# Clustering
n_clusters = 3  # Changed number of clusters to 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(features_reduced)

In [ ]:
# Visualization
plt.scatter(features_reduced[:, 0], features_reduced[:, 1], c=labels, cmap='viridis')
plt.title('Clustering of Audio Files')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar()
plt.show()

In [ ]:
# הצגת הקבצים, הקבוצות ומשך השיחה
group1=0
for idx, file in enumerate(audio_files):
    label = labels[idx]
    duration = librosa.get_duration(filename=file)
    print(f"File: {file} - Cluster: {label} - Duration: {duration:.2f} seconds")
    if label==0 :
       group1+=1
print(group1)

In [29]:
# חלוקת קבצי האודיו לפי התוויות שנוצרו
clustered_audio_files = defaultdict(list)
for file, label in zip(audio_files, labels):
    clustered_audio_files[label].append(file)

In [ ]:
# בחירת 7 קבצים אקראיים מכל קבוצה
num_samples_per_cluster = 7
selected_files_per_cluster = {}   
for cluster_id, files in clustered_audio_files.items():
    if len(files) <= num_samples_per_cluster:
        selected_files = files
    else:
        selected_files = random.sample(files, num_samples_per_cluster)    
    selected_files_per_cluster[cluster_id] = selected_files
# הצגת הקבצים שנבחרו והשמעתם
for cluster_id, selected_files in selected_files_per_cluster.items():
    print(f"\nCluster {cluster_id }:")
    for file in selected_files:
        display(Audio(filename=file))

In [ ]:
# Selecting cluster 0

In [32]:
file_path = r'C:\clasteriGroup3-0\clustering_results3.txt'
with open(file_path, 'w') as f:
    for idx, file in enumerate(audio_files):
        label = labels[idx]
        f.write(f"File: {file} - Cluster: {label}\n")  # כתיבת שורה לקובץ

In [ ]:
# Initialize an empty list to store file names
cluster_0_files = []
cnt = 0
# Open and read the file
with open(file_path, 'r') as file:
    for line in file:
        # Check if the line contains "Cluster: 0"
        if 'Cluster: 0' in line:
             # Extract the file name with full path
            full_file_name = line.split(' - ')[0].replace('File: ', '').strip()
            # Get only the file name without the path
            file_name = os.path.basename(full_file_name)
            cluster_0_files.append(file_name)
            cnt += 1

# Print the list of files in Cluster 0
print(cluster_0_files)

In [ ]:
# TESS cleaning

In [34]:
# על כל הקבצים
def remove_silence(input_folder, output_folder, silence_threshold=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        if os.path.isdir(file_path):
            new_output_folder = os.path.join(output_folder, filename)
            remove_silence(file_path, new_output_folder, silence_threshold)
        elif filename.endswith(('.wav', '.mp3')):
            try:
                # Read the file
                y, sr = librosa.load(file_path, sr=None)

                # Detect and remove silence
                non_silent_intervals = librosa.effects.split(y, top_db=silence_threshold)
                if len(non_silent_intervals) > 0:
                    non_silent_audio = np.concatenate([y[start:end] for start, end in non_silent_intervals])
                else:
                    non_silent_audio = y  # No silence detected, keep original

                # Save the file
                output_path = os.path.join(output_folder, filename)
                sf.write(output_path, non_silent_audio, sr)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Paths to source and destination folders
input_folder = r'C:\dataverse_files'
output_folder = r'C:\noSilene'

# Call the function
remove_silence(input_folder, output_folder)

In [ ]:
# extract features

In [35]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    
    # Extract Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)
    
    # Extract Mel-spectrogram features
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_mean = np.mean(mel.T, axis=0)

    # Concatenate all features
    features = np.concatenate((mfcc_mean, chroma_mean, mel_mean))

    return features

In [36]:
# List of all possible sentiments
sentiments_list = ['angry', 'boredom', 'disgust', 'fear', 'happy', 'sad', 'neutral']

In [37]:
features = []
audio_files = []
valid_files = []  # Define valid_files
sentiments = []
main_dir = r'C:\noSilene'
cnt = 0
for root, dirs, files in os.walk(main_dir):
    for file in files:
        if file.endswith(('.wav', '.mp3', '.flac')):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path)
            sentiment = file_path.split('_')[-1].replace('.wav', '')
            sentiments.append(sentiment)
            audio_files.append(file_path)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                valid_files.append(file_path)
            cnt+=1
features = np.array(features)
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(np.array(sentiments).reshape(-1, 1))

In [ ]:
y_numeric = np.argmax(encoded_data, axis=1)

# חלוקת הנתונים לנתוני אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(features, y_numeric, test_size=0.2, random_state=42)
# print("X")
# print(X_train, X_test)
# print("Y")
# print(y_train, y_test)
# יצירת המודל
model = RandomForestClassifier(n_estimators=100, random_state=42)

# אימון המודל
model.fit(X_train, y_train)

# חיזוי על נתוני הבדיקה
y_pred = model.predict(X_test)

# חישוב Confusion Matrix ודיוק
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# חישוב MSE ו-R Squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy}")
print(f"MSE: {mse}")
print(f"R Squared: {r2}")

In [ ]:
from collections import Counter
# הכנת הנתונים החדשים
recordings_path = r'C:\09'
recordings_features = []
recordings_filenames = []
for root, dirs, files in os.walk(recordings_path):
    for file in files:
        if file in cluster_0_files and file.endswith(('.wav', '.mp3', '.flac')):
            full_path = os.path.join(root, file)
            feature = extract_features(full_path)
            if feature is not None:
                recordings_features.append(feature)
                recordings_filenames.append(full_path)

# חיזוי על הנתונים החדשים
new_predictions = model.predict(recordings_features)

# שחזור המחלקות המפורשות
predicted_labels = encoder.inverse_transform(np.eye(len(model.classes_))[new_predictions])
label_counts = Counter(label[0] for label in predicted_labels)

# הדפסת התוצאות
print("Predicted labels for new recordings:")
for filename, label in zip(recordings_filenames, predicted_labels):
    print(f"File: {filename}, Predicted Class: {label[0]}")
    display(ipd.Audio(filename))
    
print("Label distribution:")
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")